In [ ]:
from flask import Flask, request, jsonify
from PIL import Image
import base64
from pyngrok import ngrok
import pytesseract
from flask_cors import CORS
from io import BytesIO
import re

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}}, methods={"GET"})
port_no = 5000
ngrok.set_auth_token("2Mx58C9YQfaH6GkAZIhxbytjKO6_4g1ymgJBg1JKNXqf4ScED")
public_url = ngrok.connect(port_no).public_url

@app.route('/upload-image', methods=['POST'])
def upload_image():

    # Nhận ảnh và trả ảnh về text
    file_str = request.form['image']
    image_bytes = base64.b64decode(file_str)
    image_re = Image.open(BytesIO(image_bytes))
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    text = pytesseract.image_to_string(image_re, lang='vie')
    print(text)

    # Tách cách dùng
    pattern = r"(.*\bsáng\b.*|.*\btrưa\b.*|.*\btối\b.*|.*\bSáng\b.*|.*\bTrưa\b.*|.*\bTối\b.*|.*\blần\b.*|.*\bSang\b.*|.*\bTrua\b.*|.*\bToi\b.*|.*\bsang\b.*|.*\btrua\b.*|.*\btoi\b.*|.*\bsóng\b.*|.*\bsúng\b.*|.*\bsung\b.*|.*\bsong\b.*)"
    pharmacies = re.findall(pattern, text)
    print("Cách dùng: ", pharmacies)

    # Tách tên thuốc
    lines = text.splitlines()
    lines = [line.strip() for line in lines if len(line.strip()) > 0]
    drugs_name = []
    for i, line in enumerate(lines):
        if any(keyword in line for keyword in ['sáng', 'trưa', 'tối', 'Sáng', 'Trưa', 'Tối', 'lần']):
            drug_line = re.sub(r"\b\w{2} Viên\b", "", lines[i-1])
            drug_name = re.sub(r'^[\.\W\d]+', '', drug_line)
            drugs_name.append(drug_name)
    print("Tên thuốc: ", drugs_name)

    # Gán vào file json
    usage = []
    i = 0
    for pharmacy in pharmacies:
        sang = ''
        trua = ''
        toi = ''
        pharmacy_words = pharmacy.lower().split()
        sang = next((pharmacy_words[i+1] for i, word in enumerate(pharmacy_words) if word in ['sáng', 'sang', 'sung', 'song', 'súng', 'sóng']), '')
        trua = next((pharmacy_words[i+1] for i, word in enumerate(pharmacy_words) if word in ['trưa', 'trua']), '')
        toi = next((pharmacy_words[i+1] for i, word in enumerate(pharmacy_words) if word in ['tối', 'chiều', 'toi', 'tôi', 'chieu']), '')

        values_to_check = [']', '[', 'I', 'i', 'J', 'j', 'Í', 'í', 'Ì', 'ì', 'Ï', '|', 'l', '/', '1]']
        if sang in values_to_check: sang = '1'
        if trua in values_to_check: trua = '1'
        if toi in values_to_check: toi = '1'
        if sang == '': sang = '0'
        if trua == '': trua = '0'
        if toi == '': toi = '0'

        usage.append({
            "name": drugs_name[i],
            "usage": {
                "Morning": {
                    "quantity": sang,
                },
                "Evening": {
                    "quantity": toi,
                },
                "Afternoon": {
                    "quantity": trua,
                }
            }
        })
        i+=1

    response = usage
    print(response)
    return jsonify(response), 200

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)